In [1]:
include("../output.jl")
using ImpvolOutput, CSV, JLD2, FileIO, Plots
gr()
default(fmt = :png)
parameters = load("../experiments/baseline/common_parameters.jld2")["parameters"]

Dict{Symbol,Any} with 45 entries:
  :S_nt                     => [0.0 0.0 … 0.0 0.0]…
  :one_over_rho             => 0.0
  :inner_tolerance          => 0.0002
  :S_nt_data                => [1184.95 -1239.14 … -406.84 1074.1]…
  :eta                      => 4.0
  :S                        => 101
  :sigma                    => 0.999
  :d                        => [0.991383 7.85e-7 … 0.000689 0.0004425; 7.42e-5 …
  :adjustment_tolerance     => 0.0004
  :middle_step_size         => 0.449329
  :A_njs                    => Array{Float64,4}[[1466.01 503.068 … 1568.37 7165…
  :gamma_jk                 => [0.17112 0.273944 … 0.0195213 0.0108426; 0.04826…
  :p_sectoral               => [1.26077 2.37256 … 1.79398 1.0]…
  :B_j                      => [5.5087]…
  :max_iter_adjustment      => 100
  :w_njt                    => [53118.2 20374.8 … 1.48903e5 1.12031e6]…
  :global_sectoral_shock    => [7.02105e-15]…
  :country_shock_njs        => Array{Float64,4}[[-2.70302e-16 -4.55127e-16 … 7.…
  :idi

In [2]:
function load_GDP(scenario)
    if scenario=="data"
        data = load("../data/impvol_data.jld2")
        cpi = CSV.read("../data/cpi.csv")
        us_cpi = reshape(cpi[:,end] ./ cpi[1,end], (1,1,1,36))
        dollar_price_index = data["pwt"] .* us_cpi
        return (sum(data["va"], 3) ./ dollar_price_index)[1,:,1,:]'
    else
        results = load("../experiments/$scenario/results.jld2")["results"]
        return sum(ImpvolOutput.make_series(results, :real_GDP), 3)[1,:,1,:]'
    end
end
function load_import(scenario)
    results = load("../experiments/$scenario/results.jld2")["results"]
    expenditure_mjt = ImpvolOutput.make_series(results, :E_mjs)
    trade_shares_mnjt = ImpvolOutput.make_series(results, :d_mnjs)
    N, _, J, T = size(trade_shares_mnjt)
    # zero out domestic spending so as not to count in imports
    for n=1:N, j=1:J, t=1:T
        trade_shares_mnjt[n,n,j,t] = 0.0
    end
    import_mjt = sum(trade_shares_mnjt, 2) .* expenditure_mjt
    return sum(import_mjt, 3)[:,1,1,:]'
end
function load_expenditure(scenario)
    results = load("../experiments/$scenario/results.jld2")["results"]
    expenditure_mjt = ImpvolOutput.make_series(results, :E_mjs)
    return sum(expenditure_mjt, 3)[:,1,1,:]'
end

load_expenditure (generic function with 1 method)

In [3]:
function compare_trade(scenarios, labels, country)
    country_names = CSV.read("../experiments/baseline/output_table.csv")[:country_names]
    X = zeros(36, length(scenarios))
    if country==0
        name = "World"
    else
        name = country_names[country]
    end
    for s = 1:length(scenarios)
        expenditure = load_expenditure(scenarios[s])
        import_value = load_import(scenarios[s])
        if country==0
            X[:,s] .= (sum(import_value, 2) ./ sum(expenditure, 2) * 100)[:,1]
        else
            X[:,s] .= import_value[:,country] ./ expenditure[:,country] * 100
        end
    end
    plot(1972:2007, X, label=labels, title=name, ylabel="Share of imports in expenditure (percent)")
end

compare_trade (generic function with 1 method)

In [4]:
function compare(scenarios, country, detrend=false)
    country_names = CSV.read("../experiments/baseline/output_table.csv")[:country_names]
    X = zeros(36, length(scenarios))
    if country==0
        name = "World"
    else
        name = country_names[country]
    end
    for s = 1:length(scenarios)
        GDP = log.(load_GDP(scenarios[s]))
        if detrend
          GDP, _ = ImpvolOutput.DetrendUtilities.detrend(GDP',parameters[:bp_weights])
            GDP = GDP'
        end
        if country==0
            X[:,s] .= log.(sum(exp.(GDP), 2)[:,1])
        else
            X[:,s] .= GDP[:,country]
        end
        println(var(X[:,s]))
    end
    plot(X, label=scenarios, title=name)
end

compare (generic function with 2 methods)

In [15]:
compare_trade(["baseline/actual", "baseline/kappa1972"], ["Baseline", "Constant trade cost"], 0)
savefig("../output/trade_share.pdf")